In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Cargar los datos desde el archivo CSV
data = pd.read_csv("Zernike.csv")

# Dividir los datos en características (features) y etiquetas (labels)
X = data.drop("etiqueta", axis=1)  # Características
y = data["etiqueta"]  # Etiquetas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# print(X_train_scaled)
print(X_train_scaled.shape)

(2560, 25)


In [2]:
learning_rate = 0.1  # Tasa de aprendizaje
momentum = 0.7  # Momentum
hidden_layers = (10, 4)  # Número de capas ocultas y número de neuronas en cada capa
activation_function = 'logistic'  # Función de activación (sigmoide)
tolerance = 0.000001 # Parámetro de tolerancia

# Crear el clasificador MLP
mlp_classifier = MLPClassifier(hidden_layer_sizes=hidden_layers,
                               activation=activation_function,
                               learning_rate_init=learning_rate,
                               momentum=momentum,
                               max_iter=1000,
                               tol=tolerance,
                               random_state=42,
                               verbose=True)

# validation_fraction=0.1,

# Entrenar el clasificador
history = mlp_classifier.fit(X_train_scaled, y_train)

Iteration 1, loss = 1.19416816
Iteration 2, loss = 1.12698634
Iteration 3, loss = 1.11557513
Iteration 4, loss = 1.10996594
Iteration 5, loss = 1.10386855
Iteration 6, loss = 1.10071318
Iteration 7, loss = 1.09524034
Iteration 8, loss = 1.09299061
Iteration 9, loss = 1.08922978
Iteration 10, loss = 1.08434425
Iteration 11, loss = 1.08148038
Iteration 12, loss = 1.08001103
Iteration 13, loss = 1.07813068
Iteration 14, loss = 1.06874057
Iteration 15, loss = 1.06443939
Iteration 16, loss = 1.06229411
Iteration 17, loss = 1.06002000
Iteration 18, loss = 1.05935725
Iteration 19, loss = 1.05932272
Iteration 20, loss = 1.04900947
Iteration 21, loss = 1.04631588
Iteration 22, loss = 1.05198744
Iteration 23, loss = 1.04820830
Iteration 24, loss = 1.04299913
Iteration 25, loss = 1.04150603
Iteration 26, loss = 1.03553156
Iteration 27, loss = 1.03630985
Iteration 28, loss = 1.03008372
Iteration 29, loss = 1.02980923
Iteration 30, loss = 1.02775385
Iteration 31, loss = 1.02734612
Iteration 32, los

In [3]:
# Evaluar el modelo
train_accuracy = mlp_classifier.score(X_train_scaled, y_train)
test_accuracy = mlp_classifier.score(X_test_scaled, y_test)

print(f"Precisión en el conjunto de entrenamiento: {train_accuracy}")
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

Precisión en el conjunto de entrenamiento: 0.578515625
Precisión en el conjunto de prueba: 0.4625


In [4]:
# Obtener las métricas de precisión durante el entrenamiento
train_accuracy_history = mlp_classifier.validation_scores_
# test_accuracy_history = [mlp_classifier.score(X_test_scaled, y_test)] * len(train_accuracy_history)

print(train_accuracy_history)

None


In [5]:
# Graficar la precisión del modelo durante el entrenamiento y la prueba
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(train_accuracy_history) + 1), train_accuracy_history, label='Train Accuracy')
plt.plot(np.arange(1, len(train_accuracy_history) + 1), test_accuracy_history, label='Test Accuracy')
plt.title('Accuracy vs Iterations')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

TypeError: object of type 'NoneType' has no len()

<Figure size 1000x600 with 0 Axes>